In [1]:
# import libraries
import os
from os.path import isfile, isdir, join
import numpy as np
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import re
from IPython.display import display
from zipfile import ZipFile
import pickle
import unicodedata
import pytz
from joblib import Parallel, delayed
import shutil
import difflib
import random
import math
from shutil import copyfile
import itertools

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler

import matplotlib as mpl
from matplotlib import pyplot as plt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

!pip install cvxpy
import cvxpy as cp

import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 2.8 MB 4.2 MB/s 
     |████████████████████████████████| 10.7 MB 48.1 MB/s 
     |████████████████████████████████| 295 kB 68.1 MB/s 
     |████████████████████████████████| 1.1 MB 69.4 MB/s 
(CVXPY) May 15 05:36:38 AM: Encountered unexpected exception importing solver GLOP:
AttributeError("module 'ortools' has no attribute '__version__'")
(CVXPY) May 15 05:36:38 AM: Encountered unexpected exception importing solver PDLP:
AttributeError("module 'ortools' has no attribute '__version__'")


In [2]:
# log
def log(msg):
    now = datetime.strftime(datetime.now(tz=pytz.timezone('Hongkong')), '%Y-%m-%d %H:%M:%S')
    print(f'[{now}] {msg}')
    
# pickle
def save_pkl(obj, filename):
    pickle.dump(obj, open(filename, 'wb'), protocol=pickle.HIGHEST_PROTOCOL)
    return
def load_pkl(filename):
    return pickle.load(open(filename, 'rb'))

def get_size(path='.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def dl_txt(full_path):
    copyfile(full_path, full_path.split('/')[-1])
    print(f"Copied {full_path.split('/')[-1]}")
    return

def clear_output():
    for file in os.listdir():
        os.remove(file)
    return

fig_num = 0
def new_plot():
    global fig_num
    fig_num += 1
    plt.figure(fig_num)
    
import signal as signal_
class TimeoutException(Exception):   # Custom exception class
    pass
def timeout_handler(signum, frame):   # Custom signal handler
    raise TimeoutException
signal_.signal(signal_.SIGALRM, timeout_handler)

# function to remove any rows/columns with all NaN
def df_drop_na(df):
    df = df.loc[lambda x: x.notnull().sum(axis=1) > 0]
    df = df[df.notnull().sum(axis=0).loc[lambda x: x>0].index.tolist()]
    return df

def align_index(dfs):
    for i in range(len(dfs)):
        if i==0:
            idx, col = dfs[i].index, dfs[i].columns
        else:
            idx, col = idx & dfs[i].index, col & dfs[i].columns
    idx, col = idx.sort_values().tolist(), col.sort_values().tolist()
    new_dfs = tuple([df.reindex(index=idx, columns=col) for df in list(dfs)])
    return new_dfs

In [3]:
# params
params = dict()
params['bt_start_date'] = '2014-01-01' # bt - back testing
params['bt_end_date'] = '2018-03-27'
params['seed'] = 0
horizons = {'6m':21*6, '9m':21*9, '12m':21*12}

In [4]:
# load returns
ret = pd.read_csv('../input/hkml-download-returns/ret.csv').assign(date = lambda x: pd.to_datetime(x.date)).set_index('date')
exret = pd.read_csv('../input/hkml-download-returns/exret.csv').assign(date = lambda x: pd.to_datetime(x.date)).set_index('date')

# filter returns to testing period
exret = exret.loc[lambda x: (x.index>=params['bt_start_date']) & (x.index<=params['bt_end_date'])]
exret = df_drop_na(exret)
ret, exret = align_index((ret, exret))
log(f'Shape of ret: {ret.shape}')
log(f'Shape of exret: {exret.shape}')

# load 10-K signals
feats_10k = pd.merge(pd.read_csv('../input/hkml-signal-extraction-10k-cpu/feats.csv'),
                 pd.read_csv('../input/hkml-signal-extraction-gpu/feats.csv'),
                 how='inner', on=['doc_id','cik','entity','filing_date','stock']) \
    .rename(columns={'filing_date':'date'}) \
    .assign(date = lambda x: pd.to_datetime(x.date),
            cik = lambda x: x.cik.astype(str).str.zfill(10)) \
    .replace([np.inf, -np.inf], np.nan)
feat_names = [c for c in feats_10k.columns if 'feat' in c]
feats_10k = feats_10k.rename(columns={c:c+'_10k' for c in feat_names})
feats_10k = feats_10k.loc[:,['stock','date'] + [c for c in feats_10k.columns if 'feat' in c]]
log(f'Shape of 10-K feats: {feats_10k.shape}')

# load 10-Q signals
feats_10q = pd.read_csv('../input/hkml-signal-extraction-10q-cpu/feats.csv') \
    .rename(columns={'filing_date':'date'}) \
    .assign(date = lambda x: pd.to_datetime(x.date),
            cik = lambda x: x.cik.astype(str).str.zfill(10)) \
    .replace([np.inf, -np.inf])
feat_names = [c for c in feats_10q.columns if 'feat' in c]
feats_10q = feats_10q.rename(columns={c:c+'_10q' for c in feat_names})
feats_10q = feats_10q.loc[:,['stock','date'] + [c for c in feats_10q.columns if 'feat' in c]]
log(f'Shape of 10-Q feats: {feats_10q.shape}')

# load 8-k signal
feats_8k = load_pkl('../input/hkml-signal-extraction-pre/feats_8k')
log(f'Shape of 8-K feats: {feats_8k.shape}')

# load LTR signal
feats_lgbm_ltr = load_pkl(f'../input/hkml-lightgbm-ltr/pred_val_out').rename(columns={0:'feat_lgbm_ltr_12m'})
log(f'Shape of LGBM Learning-to-Rank feats: {feats_lgbm_ltr.shape}')

# load LTR signal
feats_lgbm_binary_clf = load_pkl(f'../input/hkml-lightgbm-binary-clf/pred_prob_test_out')
log(f'Shape of LGBM binary classifier feats: {feats_lgbm_binary_clf.shape}')

# combine all signals into single df
feats = feats_10k \
    .merge(feats_10q, how='outer', on=['stock','date']) \
    .merge(feats_8k, how='outer', on=['stock','date']) \
    .merge(feats_lgbm_ltr, how='outer', on=['stock','date']) \
    .merge(feats_lgbm_binary_clf, how='outer', on=['stock','date']) \
    .sort_values(['stock','date']) \
    .groupby('stock') \
    .apply(lambda x: x.ffill()) \
    .loc[lambda x: (x.date>=params['bt_start_date']) & (x.date<=params['bt_end_date'])] \
    .reset_index(drop=True)

[2022-05-15 13:36:40] Shape of ret: (1064, 596)
[2022-05-15 13:36:40] Shape of exret: (1064, 596)
[2022-05-15 13:36:40] Shape of 10-K feats: (5219, 19)
[2022-05-15 13:36:40] Shape of 10-Q feats: (13457, 7)
[2022-05-15 13:36:41] Shape of 8-K feats: (1261848, 4)
[2022-05-15 13:36:41] Shape of LGBM Learning-to-Rank feats: (639899, 3)
[2022-05-15 13:36:41] Shape of LGBM binary classifier feats: (861573, 3)


In [5]:
# summary DQ
feat_names = [c for c in feats.columns if 'feat' in c]
log(f'Shape of combined feats: {feats.shape}')
display(feats.head())

[2022-05-15 13:36:46] Shape of combined feats: (647996, 28)


,stock,date,feat_ch_full_len_10k,feat_ch_item_1a_len_10k,feat_ch_item_1b_len_10k,feat_ch_item_3_len_10k,feat_full_cos_1gram_10k,feat_full_cos_2gram_10k,feat_full_jac_1gram_10k,feat_full_jac_2gram_10k,feat_item_1a_lev_10k,feat_item_7_lev_10k,feat_lm_postive_10k,feat_lm_uncertainty_10k,feat_lm_litigious_10k,feat_word2vec_10k,feat_sen_enc_10k,feat_item_sentiment_10k,feat_fls_sentiment_10k,feat_ch_full_len_10q,feat_full_cos_1gram_10q,feat_full_jac_1gram_10q,feat_word2vec_10q,feat_lm_postive_10q,feat_cnt_8k,feat_cnt_8k_diff,feat_lgbm_ltr_12m,feat_lgbm_binary_clf_12m
0,A,2014-01-02,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,236.0,0.527291
1,A,2014-01-03,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,237.0,0.527291
2,A,2014-01-06,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,237.0,0.527291
3,A,2014-01-07,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,237.0,0.527291
4,A,2014-01-08,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,237.0,0.527291


In [6]:
def get_portfolio_ret(signal, f_ret, n_day, div_vol=False):
    ranks = signal.rank(axis=1) - 1 / 2
    weights = 2 * ranks.divide(ranks.count(axis=1), axis=0) - 1
    std = f_ret.shift(n_day).rolling(252, 252//2).std() if div_vol else 1
    port_ret = (weights / std * f_ret).sum(axis=1)
    return port_ret


def get_sharpe(port, n_day):
    return port.mean() * np.sqrt(252/n_day) / port.std()


def gen_metric(signal, ret, exret, n_day):
    # future returns
    f_ret = (1+ret).rolling(n_day).apply(np.prod, raw=True).shift(-n_day) - 1
    f_exret = (1+exret).rolling(n_day).apply(np.prod, raw=True).shift(-n_day) - 1
    f_ret, f_exret = df_drop_na(f_ret), df_drop_na(f_exret)
    signal, f_ret, f_exret = align_index((signal, f_ret, f_exret))
    signal = signal.mask(f_ret.isnull())
    # ranked signal, returns
    signal_rnk, f_ret_rnk, f_exret_rnk = signal.rank(axis=1), f_ret.rank(axis=1), f_exret.rank(axis=1)
    # average correlation between signal and excess return (both ranked)
    avg_rnk_corr = pd.Series(signal_rnk.index).apply(lambda x: pd.concat([signal_rnk.loc[x], f_exret_rnk.loc[x]], axis=1).corr().iloc[0,1]).mean()
    # construct uniform weight portfolio
    port_ret = get_portfolio_ret(signal, f_ret, n_day)
    port_exret = get_portfolio_ret(signal, f_exret, n_day)
    port_ret_vol = get_portfolio_ret(signal, f_ret, n_day, div_vol=True)
    # calculate sharpe ratios
    sharpe_ret = get_sharpe(port_ret, n_day)
    sharpe_exret = get_sharpe(port_exret, n_day)
    sharpe_ret_vol = get_sharpe(port_ret_vol, n_day)
    return avg_rnk_corr, sharpe_exret, sharpe_ret, sharpe_ret_vol


def analyze_feat(feats, ret, exret, selected_feat):
    # selected_feat = 'feat_full_jac_2gram'

    # create signal table as pivot
    signal = feats[['stock', 'date', selected_feat]] \
        .drop_duplicates() \
        .pivot('date', 'stock', selected_feat)

    # forward-fill and align index with returns
    dates = (signal.index | ret.index).sort_values().tolist()
    signal = signal.reindex(index=dates).ffill()
    signal = df_drop_na(signal)
    signal, ret_, exret_ = align_index((signal, ret, exret))

    log(f'Shape of ret: {ret_.shape}')
    log(f'Shape of exret: {exret_.shape}')
    log(f'Shape of signal: {signal.shape}')

    # calculate metrics per investiment horizon
    metric = []
    for h in horizons:
        metric.append([selected_feat, h, horizons[h]] + list(gen_metric(signal, ret, exret, horizons[h])))
    metric = pd.DataFrame(metric, columns=['feat', 'horizon', 'n_day', 'avg_rnk_corr', 'sharpe_exret', 'sharpe_ret', 'sharpe_ret_vol'])
    return metric

In [7]:
# loop through all signals to generate metrics
feat_metric = [analyze_feat(feats, ret, exret, f) for f in feat_names]
feat_metric = pd.concat(feat_metric, axis=0).reset_index(drop=True)

[2022-05-15 13:36:46] Shape of ret: (1064, 557)
[2022-05-15 13:36:46] Shape of exret: (1064, 557)
[2022-05-15 13:36:46] Shape of signal: (1064, 557)
[2022-05-15 13:37:08] Shape of ret: (1064, 554)
[2022-05-15 13:37:08] Shape of exret: (1064, 554)
[2022-05-15 13:37:08] Shape of signal: (1064, 554)
[2022-05-15 13:37:31] Shape of ret: (1064, 550)
[2022-05-15 13:37:31] Shape of exret: (1064, 550)
[2022-05-15 13:37:31] Shape of signal: (1064, 550)
[2022-05-15 13:37:53] Shape of ret: (1064, 554)
[2022-05-15 13:37:53] Shape of exret: (1064, 554)
[2022-05-15 13:37:53] Shape of signal: (1064, 554)
[2022-05-15 13:38:15] Shape of ret: (1064, 557)
[2022-05-15 13:38:15] Shape of exret: (1064, 557)
[2022-05-15 13:38:15] Shape of signal: (1064, 557)
[2022-05-15 13:38:37] Shape of ret: (1064, 557)
[2022-05-15 13:38:37] Shape of exret: (1064, 557)
[2022-05-15 13:38:37] Shape of signal: (1064, 557)
[2022-05-15 13:39:00] Shape of ret: (1064, 557)
[2022-05-15 13:39:00] Shape of exret: (1064, 557)
[2022-05

In [8]:
# based on first round analysis, compute various weighted averages of signals
s_list = [0, 0.4]
k_list = [0.05, 0.10, 0.15, 0.20, -0.05, -0.10, -0.15, -0.20, 0]
t_dict = {'minmax': MinMaxScaler(),
          'uniform': QuantileTransformer(output_distribution='uniform', random_state=0),
          'normal': QuantileTransformer(output_distribution='normal', random_state=0)}

list1 = list(itertools.product(s_list, [-0.1,-0.05, 0, 0.05, 0.1], ['normal']))
list2 = list(itertools.product(s_list, [0,0.5,1,2,3], ['minmax','uniform']))

for s, k, t in list1 + list2:
    weights = feat_metric \
        .loc[lambda x: (x.horizon=='12m') & (x.sharpe_ret>s) & (~x.feat.str.contains('avg')) & (~x.feat.str.contains('lgbm'))] \
        .loc[:, ['feat','sharpe_ret']] \
        .sort_values('sharpe_ret', ascending=False)
    weights['imp'] = np.exp(k * weights['sharpe_ret'])
    weights['weight'] = weights['imp'] / np.sum(weights['imp'])
    df = feats[weights.feat.tolist()]
    df = pd.DataFrame(t_dict[t].fit_transform(df), columns=df.columns)
    df = df.multiply(weights.weight.tolist(), axis=1)
    feats[f'feat_weighted_avg_s{s}_k{k}_{t}'] = df.sum(axis=1)

# summary DQ
feat_names = [c for c in feats.columns if 'feat' in c]
log(f'Shape of combined feats: {feats.shape}')
display(feats.head())

# output combined signal
display(feats[feat_names].sum())
save_pkl(feats, 'feats')

[2022-05-15 13:46:51] Shape of combined feats: (647996, 58)


,stock,date,feat_ch_full_len_10k,feat_ch_item_1a_len_10k,feat_ch_item_1b_len_10k,feat_ch_item_3_len_10k,feat_full_cos_1gram_10k,feat_full_cos_2gram_10k,feat_full_jac_1gram_10k,feat_full_jac_2gram_10k,feat_item_1a_lev_10k,feat_item_7_lev_10k,feat_lm_postive_10k,feat_lm_uncertainty_10k,feat_lm_litigious_10k,feat_word2vec_10k,feat_sen_enc_10k,feat_item_sentiment_10k,feat_fls_sentiment_10k,feat_ch_full_len_10q,feat_full_cos_1gram_10q,feat_full_jac_1gram_10q,feat_word2vec_10q,feat_lm_postive_10q,feat_cnt_8k,feat_cnt_8k_diff,feat_lgbm_ltr_12m,feat_lgbm_binary_clf_12m,feat_weighted_avg_s0_k-0.1_normal,feat_weighted_avg_s0_k-0.05_normal,feat_weighted_avg_s0_k0_normal,feat_weighted_avg_s0_k0.05_normal,feat_weighted_avg_s0_k0.1_normal,feat_weighted_avg_s0.4_k-0.1_normal,feat_weighted_avg_s0.4_k-0.05_normal,feat_weighted_avg_s0.4_k0_normal,feat_weighted_avg_s0.4_k0.05_normal,feat_weighted_avg_s0.4_k0.1_normal,feat_weighted_avg_s0_k0_minmax,feat_weighted_avg_s0_k0_uniform,feat_weighted_avg_s0_k0.5_minmax,feat_weighted_avg_s0_k0.5_uniform,feat_weighted_avg_s0_k1_minmax,feat_weighted_avg_s0_k1_uniform,feat_weighted_avg_s0_k2_minmax,feat_weighted_avg_s0_k2_uniform,feat_weighted_avg_s0_k3_minmax,feat_weighted_avg_s0_k3_uniform,feat_weighted_avg_s0.4_k0_minmax,feat_weighted_avg_s0.4_k0_uniform,feat_weighted_avg_s0.4_k0.5_minmax,feat_weighted_avg_s0.4_k0.5_uniform,feat_weighted_avg_s0.4_k1_minmax,feat_weighted_avg_s0.4_k1_uniform,feat_weighted_avg_s0.4_k2_minmax,feat_weighted_avg_s0.4_k2_uniform,feat_weighted_avg_s0.4_k3_minmax,feat_weighted_avg_s0.4_k3_uniform
0,A,2014-01-02,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,236.0,0.527291,-0.199322,-0.190936,-0.182292,-0.173387,-0.164218,0.174803,0.181425,0.188138,0.194938,0.201821,0.660996,0.456432,0.678704,0.488549,0.704022,0.52966,0.766086,0.619521,0.808923,0.676174,0.714022,0.563569,0.730947,0.5901,0.752083,0.618068,0.793892,0.665844,0.818243,0.690923
1,A,2014-01-03,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,237.0,0.527291,-0.199322,-0.190936,-0.182292,-0.173387,-0.164218,0.174803,0.181425,0.188138,0.194938,0.201821,0.660996,0.456432,0.678704,0.488549,0.704022,0.52966,0.766086,0.619521,0.808923,0.676174,0.714022,0.563569,0.730947,0.5901,0.752083,0.618068,0.793892,0.665844,0.818243,0.690923
2,A,2014-01-06,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,237.0,0.527291,-0.199322,-0.190936,-0.182292,-0.173387,-0.164218,0.174803,0.181425,0.188138,0.194938,0.201821,0.660996,0.456432,0.678704,0.488549,0.704022,0.52966,0.766086,0.619521,0.808923,0.676174,0.714022,0.563569,0.730947,0.5901,0.752083,0.618068,0.793892,0.665844,0.818243,0.690923
3,A,2014-01-07,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,237.0,0.527291,-0.199322,-0.190936,-0.182292,-0.173387,-0.164218,0.174803,0.181425,0.188138,0.194938,0.201821,0.660996,0.456432,0.678704,0.488549,0.704022,0.52966,0.766086,0.619521,0.808923,0.676174,0.714022,0.563569,0.730947,0.5901,0.752083,0.618068,0.793892,0.665844,0.818243,0.690923
4,A,2014-01-08,0.003344,-0.120401,-0.0,-1.781048,0.96672,0.920556,0.896534,0.787751,0.820244,0.807044,-0.000257,-0.000106,-0.000098,0.999092,0.998004,-0.050531,-0.020592,-0.071944,0.99643,0.832324,0.99801,-0.000593,-10.0,1.0,237.0,0.527291,-0.199322,-0.190936,-0.182292,-0.173387,-0.164218,0.174803,0.181425,0.188138,0.194938,0.201821,0.660996,0.456432,0.678704,0.488549,0.704022,0.

feat_ch_full_len_10k                   -9.995012e+03
feat_ch_item_1a_len_10k                -2.179835e+04
feat_ch_item_1b_len_10k                 4.624800e+03
feat_ch_item_3_len_10k                  5.643059e+04
feat_full_cos_1gram_10k                 5.684832e+05
feat_full_cos_2gram_10k                 5.519919e+05
feat_full_jac_1gram_10k                 5.161512e+05
feat_full_jac_2gram_10k                 4.408573e+05
feat_item_1a_lev_10k                    4.978536e+05
feat_item_7_lev_10k                     4.737510e+05
feat_lm_postive_10k                    -1.236898e+01
feat_lm_uncertainty_10k                -5.436695e+00
feat_lm_litigious_10k                   3.817757e-01
feat_word2vec_10k                       5.952284e+05
feat_sen_enc_10k                        5.935209e+05
feat_item_sentiment_10k                -1.530911e+03
feat_fls_sentiment_10k                 -1.667474e+03
feat_ch_full_len_10q                   -1.133872e+04
feat_full_cos_1gram_10q                 6.0156

In [9]:
# loop through all signals to generate metrics
feat_avg_names = [c for c in feats.columns if 'feat_weighted_avg' in c]
feat_metric_avg = [analyze_feat(feats, ret, exret, f) for f in feat_avg_names]
feat_metric_avg = pd.concat(feat_metric_avg, axis=0).reset_index(drop=True)
feat_metric = pd.concat([feat_metric, feat_metric_avg], axis=0).reset_index(drop=True)
feat_metric.to_csv('feat_metric.csv', index=False)

[2022-05-15 13:46:53] Shape of ret: (1064, 560)
[2022-05-15 13:46:53] Shape of exret: (1064, 560)
[2022-05-15 13:46:53] Shape of signal: (1064, 560)
[2022-05-15 13:47:16] Shape of ret: (1064, 560)
[2022-05-15 13:47:16] Shape of exret: (1064, 560)
[2022-05-15 13:47:16] Shape of signal: (1064, 560)
[2022-05-15 13:47:39] Shape of ret: (1064, 560)
[2022-05-15 13:47:39] Shape of exret: (1064, 560)
[2022-05-15 13:47:39] Shape of signal: (1064, 560)
[2022-05-15 13:48:01] Shape of ret: (1064, 560)
[2022-05-15 13:48:01] Shape of exret: (1064, 560)
[2022-05-15 13:48:01] Shape of signal: (1064, 560)
[2022-05-15 13:48:23] Shape of ret: (1064, 560)
[2022-05-15 13:48:23] Shape of exret: (1064, 560)
[2022-05-15 13:48:23] Shape of signal: (1064, 560)
[2022-05-15 13:48:46] Shape of ret: (1064, 560)
[2022-05-15 13:48:46] Shape of exret: (1064, 560)
[2022-05-15 13:48:46] Shape of signal: (1064, 560)
[2022-05-15 13:49:08] Shape of ret: (1064, 560)
[2022-05-15 13:49:08] Shape of exret: (1064, 560)
[2022-05

In [10]:
# display all signal metric output
feat_names = [c for c in feats.columns if 'feat' in c]
for feat in feat_names:
    display(feat_metric.loc[lambda x: x.feat==feat])

,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
0,feat_ch_full_len_10k,6m,126,0.011788,0.410710,0.515007,0.384209
1,feat_ch_full_len_10k,9m,189,0.027996,0.665152,0.686653,0.733436
2,feat_ch_full_len_10k,12m,252,0.039372,0.827734,0.776643,0.835370


,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
3,feat_ch_item_1a_len_10k,6m,126,0.012262,0.417152,0.268661,0.435046
4,feat_ch_item_1a_len_10k,9m,189,0.021461,0.431898,0.290181,0.515214
5,feat_ch_item_1a_len_10k,12m,252,0.032616,0.504538,0.349152,0.870880


,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
6,feat_ch_item_1b_len_10k,6m,126,-0.019051,-0.508632,-0.444818,-0.780796
7,feat_ch_item_1b_len_10k,9m,189,-0.015362,-0.451767,-0.368533,-0.673344
8,feat_ch_item_1b_len_10k,12m,252,-0.009736,-0.289043,-0.272941,-0.322770


,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
9,feat_ch_item_3_len_10k,6m,126,-0.008015,-0.384300,-0.454176,-0.493154
10,feat_ch_item_3_len_10k,9m,189,-0.003299,-0.301118,-0.454885,-0.566054
11,feat_ch_item_3_len_10k,12m,252,0.000248,-0.114657,-0.246312,-0.118548


,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
12,feat_full_cos_1gram_10k,6m,126,-0.004400,-0.038721,-0.142918,0.408696
13,feat_full_cos_1gram_10k,9m,189,-0.007394,-0.151657,-0.245219,0.535060
14,feat_full_cos_1gram_10k,12m,252,-0.009598,-0.224323,-0.305044,0.663217


...

,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
153,feat_weighted_avg_s0.4_k1_uniform,6m,126,0.064651,1.306861,1.323557,0.872857
154,feat_weighted_avg_s0.4_k1_uniform,9m,189,0.090050,1.645441,1.712431,0.653241
155,feat_weighted_avg_s0.4_k1_uniform,12m,252,0.106225,1.848528,2.019676,0.717011


,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
156,feat_weighted_avg_s0.4_k2_minmax,6m,126,0.061071,1.191455,1.215955,0.854350
157,feat_weighted_avg_s0.4_k2_minmax,9m,189,0.083035,1.480369,1.566072,0.562264
158,feat_weighted_avg_s0.4_k2_minmax,12m,252,0.096569,1.694656,1.914352,0.674362


,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
159,feat_weighted_avg_s0.4_k2_uniform,6m,126,0.063526,1.245046,1.263638,0.844830
160,feat_weighted_avg_s0.4_k2_uniform,9m,189,0.087131,1.557423,1.636979,0.593139
161,feat_weighted_avg_s0.4_k2_uniform,12m,252,0.101092,1.763872,1.971272,0.683100


,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
162,feat_weighted_avg_s0.4_k3_minmax,6m,126,0.060039,1.178564,1.198972,0.823003
163,feat_weighted_avg_s0.4_k3_minmax,9m,189,0.081209,1.453920,1.538295,0.542891
164,feat_weighted_avg_s0.4_k3_minmax,12m,252,0.093816,1.644078,1.859264,0.656731


,feat,horizon,n_day,avg_rnk_corr,sharpe_exret,sharpe_ret,sharpe_ret_vol
165,feat_weighted_avg_s0.4_k3_uniform,6m,126,0.061888,1.212381,1.229577,0.836859
166,feat_weighted_avg_s0.4_k3_uniform,9m,189,0.084001,1.505562,1.585987,0.566693
167,feat_weighted_avg_s0.4_k3_uniform,12m,252,0.096825,1.705747,1.919094,0.665988


In [11]:
# signal correlation plot
corr = feats[feat_names].corr()
corr.style.background_gradient(cmap='coolwarm')

,feat_ch_full_len_10k,feat_ch_item_1a_len_10k,feat_ch_item_1b_len_10k,feat_ch_item_3_len_10k,feat_full_cos_1gram_10k,feat_full_cos_2gram_10k,feat_full_jac_1gram_10k,feat_full_jac_2gram_10k,feat_item_1a_lev_10k,feat_item_7_lev_10k,feat_lm_postive_10k,feat_lm_uncertainty_10k,feat_lm_litigious_10k,feat_word2vec_10k,feat_sen_enc_10k,feat_item_sentiment_10k,feat_fls_sentiment_10k,feat_ch_full_len_10q,feat_full_cos_1gram_10q,feat_full_jac_1gram_10q,feat_word2vec_10q,feat_lm_postive_10q,feat_cnt_8k,feat_cnt_8k_diff,feat_lgbm_ltr_12m,feat_lgbm_binary_clf_12m,feat_weighted_avg_s0_k-0.1_normal,feat_weighted_avg_s0_k-0.05_normal,feat_weighted_avg_s0_k0_normal,feat_weighted_avg_s0_k0.05_normal,feat_weighted_avg_s0_k0.1_normal,feat_weighted_avg_s0.4_k-0.1_normal,feat_weighted_avg_s0.4_k-0.05_normal,feat_weighted_avg_s0.4_k0_normal,feat_weighted_avg_s0.4_k0.05_normal,feat_weighted_avg_s0.4_k0.1_normal,feat_weighted_avg_s0_k0_minmax,feat_weighted_avg_s0_k0_uniform,feat_weighted_avg_s0_k0.5_minmax,feat_weighted_avg_s0_k0.5_uniform,feat_weighted_avg_s0_k1_minmax,feat_weighted_avg_s0_k1_uniform,feat_weighted_avg_s0_k2_minmax,feat_weighted_avg_s0_k2_uniform,feat_weighted_avg_s0_k3_minmax,feat_weighted_avg_s0_k3_uniform,feat_weighted_avg_s0.4_k0_minmax,feat_weighted_avg_s0.4_k0_uniform,feat_weighted_avg_s0.4_k0.5_minmax,feat_weighted_avg_s0.4_k0.5_uniform,feat_weighted_avg_s0.4_k1_minmax,feat_weighted_avg_s0.4_k1_uniform,feat_weighted_avg_s0.4_k2_minmax,feat_weighted_avg_s0.4_k2_uniform,feat_weighted_avg_s0.4_k3_minmax,feat_weighted_avg_s0.4_k3_uniform
feat_ch_full_len_10k,1.000000,0.236127,0.057164,0.119494,0.089829,0.066416,-0.038341,0.006713,-0.036918,0.040787,0.145966,0.199059,-0.323651,-0.267430,-0.077245,-0.184135,-0.067629,0.137331,0.017321,0.010928,0.062491,0.064683,-0.049795,0.007418,0.003906,0.065206,0.133103,0.133537,0.133883,0.134136,0.134287,0.147473,0.146872,0.146118,0.145208,0.144143,0.072870,0.138651,0.071856,0.131767,0.060435,0.111692,0.031734,0.057879,0.017209,0.026442,0.106443,0.154236,0.087467,0.128333,0.064329,0.098341,0.030988,0.049103,0.016882,0.023966
feat_ch_item_1a_len_10k,0.236127,1.000000,0.017620,0.044747,0.021486,0.017516,-0.022813,-0.005144,-0.065097,-0.083034,0.175409,0.163327,-0.035010,-0.139140,-0.047611,-0.109811,-0.036874,0.082566,0.045371,0.043660,0.061639,0.044849,-0.007635,0.004549,-0.006108,-0.021677,0.167593,0.167224,0.166776,0.166244,0.165623,0.063558,0.064791,0.065964,0.067074,0.068119,0.099405,0.152585,0.094453,0.140092,0.081952,0.118019,0.057165,0.069982,0.046701,0.045428,0.013672,0.080069,0.029116,0.075913,0.038619,0.068114,0.044067,0.051279,0.043576,0.040678
feat_ch_item_1b_len_10k,0.057164,0.017620,1.000000,0.013936,-0.011899,-0.006902,-0.003332,0.014141,0.001761,-0.018669,0.010657,0.005741,-0.032918,-0.020068,0.001737,-0.039437,-0.091367,0.069908,-0.005259,0.015410,-0.001046,0.013723,-0.004506,0.014695,-0.000402,0.028462,0.022537,0.022577,0.022605,0.022621,0.022624,0.025889,0.025661,0.025417,0.025158,0.024883,0.000409,0.029355,0.004115,0.027127,0.007814,0.023052,0.012133,0.014962,0.013758,0.012096,0.004879,0.031340,0.008079,0.024992,0.010380,0.019401,0.012762,0.013028,0.013885,0.011578
feat_ch_item_3_len_10k,0.119494,0.044747,0.013936,1.000000,0.006001,-0.022739,-0.035712,-0.049052,0.012677,-0.003893,0.244701,-0.001561,0.202264,-0.060851,-0.028925,-0.030841,-0.034972,0.134270,0.027884,-0.027240,0.035536,0.130769,-0.011625,0.021238,-0.016915,-0.087580,0.095081,0.094923,0.094689,0.094373,0.093971,0.099345,0.098651,0.097826,0.096869,0.095781,0.051003,0.090984,0.042892,0.082361,0.027863,0.063104,-0.000831,0.014544,-0.015001,-0.014606,0.049113,0.104515,0.036819,0.082140,0.021189,0.055252,-0.003328,0.009121,-0.015574,-0.016062
feat_full_cos_1gram_10k,0.089829,0.021486,-0.011899,0.006001,1.000000,0.898812,0.488807,0.475444,0.158964,0.162709,0.033581,0.026576,-0.079196,0.347334,0.372497,0.035192,0.038019,0.079067,0.221054,0.182426,0.160587,0.017523,0.005865,-0.001632,0.101255,0.055206,0.360576,0.359221,0.3